In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model

In [2]:
# Define labels for each dataset (update with your actual labels)
dataset_labels = {
    'tricep pushdown': ['Correct', 'Incorrect'],
    'squat': ['Correct', 'Incorrect'],
    'leg raises': ['Correct', 'Incorrect'],
    'hammer curl': ['Correct', 'Incorrect']
}

In [3]:
# Prompt the user to input the dataset name
dataset_name = input("Enter the dataset name (dataset1, dataset2, dataset3, dataset4): ")

Enter the dataset name (dataset1, dataset2, dataset3, dataset4):  hammer curl


In [4]:
# Check if the dataset name is valid
if dataset_name not in dataset_labels:
    print(f"Invalid dataset name. Please choose from {list(dataset_labels.keys())}.")
    exit(1)


In [5]:
# Load the corresponding model and labels
model_path = f'PoseModel_{dataset_name}.keras'
if not os.path.exists(model_path):
    print(f"Model for {dataset_name} not found.")
    exit(1)

model = load_model(model_path)
labels = dataset_labels[dataset_name]


In [6]:
# Define the default label and confidence threshold
default_label = 'Incorrect'  # Replace with your actual default label
confidence_threshold = 0.2  # Set your confidence threshold

In [7]:
# Function to extract features from a given image using the same preprocessing as training
def extract_pose_features(image, model):
    image = cv2.resize(image, (224, 224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    preprocessed_image = preprocess_input(image)
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)

    features = model.predict(preprocessed_image)
    return features.flatten()


In [8]:
# Load the base model used for feature extraction (PoseNet in this case)
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [9]:
# Function to predict the label for the given frame using the specified dataset
def predict_label(frame, model, base_model, labels, default_label, confidence_threshold):
    # Extract features from the frame
    features = extract_pose_features(frame, base_model)
    features = np.expand_dims(features, axis=0)

    # Predict the label of the frame
    prediction = model.predict(features)
    max_confidence = np.max(prediction)
    predicted_label_index = np.argmax(prediction)

    # Check if the confidence is above the threshold
    if max_confidence >= confidence_threshold:
        predicted_label = labels[predicted_label_index]
    else:
        predicted_label = default_label

    return predicted_label


In [10]:
# Start capturing from the webcam
cap = cv2.VideoCapture(0)

In [11]:
print("Press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Predict the label for the current frame
    predicted_label = predict_label(frame, model, base_model, labels, default_label, confidence_threshold)

    # Display the label on the frame
    cv2.putText(frame, f'Label: {predicted_label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow('Webcam Feed', frame)

    # Check for 'q' key to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

Press 'q' to quit
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/s